# Modeling USDA ERS Food data

In [1]:
dataset_id = "USDA_ERS_modeled"

In [2]:
!bq --location=US mk --dataset {dataset_id}  #Note: This will not work if you already have a dataset with this name

BigQuery error in mk operation: Dataset 'responsive-cab-267123:USDA_ERS_modeled'
already exists.


The `FIPS_Market_Group` and `State_Codes` tables do not contribute anything to our planned analysis, as descriped in the DATASETS.txt file, so we don't transfer them over to our modeled dataset. The `Geo_Market_Group` table contains the same data as `Geo_Market`, so we can drop it.

To model:

* some child tables have the "name" instead of "id" as the FK. We think id should be FK 
* Drop the extra blank column(s) in `Market_Groups` table(s)
    * change column names to what's on ERD
* Add meaningul column names to `Geo_Market`: (check ERD)
    * the `nielson_name` (4th column) field in the `Geo_Master` table violates the 1NF design principle --drop it
* union `Food_#_Market` tables, add a column to represent the food# (This table needs a PK)
    * changed some attribute names in Food_#_Market (but feel free to change them back and fix ERD)
        * se to standard_error
        * n to sample_size
        * aggweight to agg_weight
        * totexp to tot_q_exp
    * deleted region and division attributes from the new unioned `Food_Market` because it's repetitive (can be traced back to Geo_Market parent table)
* anything else you can think of
* anything not transformed/modeled, add to TRANSFORMS.txt file
    * we can make the mapping table between primary and secondary dataset in the next milestone

### Copy Foods table from staging dataset

In [3]:
%%bigquery
create table USDA_ERS_modeled.Foods as select distinct * from USDA_ERS_staging.Foods

Executing query with job ID: e9c0e57f-1bcb-4d8e-8096-c011dafb6b58
Query executing: 0.56s

Conflict: 409 GET https://www.googleapis.com/bigquery/v2/projects/responsive-cab-267123/queries/e9c0e57f-1bcb-4d8e-8096-c011dafb6b58?timeoutMs=400&location=US&maxResults=0: Already Exists: Table responsive-cab-267123:USDA_ERS_modeled.Foods

### Copy Food_Categories table from staging dataset

In [4]:
%%bigquery
create table USDA_ERS_modeled.Food_Categories as select distinct * from USDA_ERS_staging.Food_Categories

Executing query with job ID: e3c35cae-f2ba-440e-9053-4cae11da3a0e
Query executing: 0.59s

Conflict: 409 GET https://www.googleapis.com/bigquery/v2/projects/responsive-cab-267123/queries/e3c35cae-f2ba-440e-9053-4cae11da3a0e?timeoutMs=400&location=US&maxResults=0: Already Exists: Table responsive-cab-267123:USDA_ERS_modeled.Food_Categories

### Create the Market_Groups table that will contain market_id and market_names

We are dropping the three string fields from the staging dataset (only need market_id and market_name)

In [ ]:
%%bigquery
create table USDA_ERS_modeled.Market_Groups as select market_id, market_name from USDA_ERS_staging.Market_Groups

### Create a table that combines all Food_#_Market Tables from staging dataset

Will use marketgroup (change name to market_id),
year, quarter, price, se (will explicitly name it as standard_error),
n (sample_size), aggweight (agg_weight), totexp (tot_q_exp),
and food_id (this will correspond to the original number specified in each staging table -> 1-54).

In [5]:
%%bigquery
create table USDA_ERS_modeled.Food_Market as 
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 1 as food_id from USDA_ERS_staging.Food_1_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 2 as food_id from USDA_ERS_staging.Food_2_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 3 as food_id from USDA_ERS_staging.Food_3_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 4 as food_id from USDA_ERS_staging.Food_4_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 5 as food_id from USDA_ERS_staging.Food_5_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 6 as food_id from USDA_ERS_staging.Food_6_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 7 as food_id from USDA_ERS_staging.Food_7_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 8 as food_id from USDA_ERS_staging.Food_8_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 9 as food_id from USDA_ERS_staging.Food_9_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 10 as food_id from USDA_ERS_staging.Food_10_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 11 as food_id from USDA_ERS_staging.Food_11_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 12 as food_id from USDA_ERS_staging.Food_12_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 13 as food_id from USDA_ERS_staging.Food_13_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 14 as food_id from USDA_ERS_staging.Food_14_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 15 as food_id from USDA_ERS_staging.Food_15_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 16 as food_id from USDA_ERS_staging.Food_16_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 17 as food_id from USDA_ERS_staging.Food_17_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 18 as food_id from USDA_ERS_staging.Food_18_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 19 as food_id from USDA_ERS_staging.Food_19_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 20 as food_id from USDA_ERS_staging.Food_20_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 21 as food_id from USDA_ERS_staging.Food_21_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 22 as food_id from USDA_ERS_staging.Food_22_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 23 as food_id from USDA_ERS_staging.Food_23_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 24 as food_id from USDA_ERS_staging.Food_24_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 25 as food_id from USDA_ERS_staging.Food_25_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 26 as food_id from USDA_ERS_staging.Food_26_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 27 as food_id from USDA_ERS_staging.Food_27_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 28 as food_id from USDA_ERS_staging.Food_28_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 29 as food_id from USDA_ERS_staging.Food_29_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 30 as food_id from USDA_ERS_staging.Food_30_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 31 as food_id from USDA_ERS_staging.Food_31_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 32 as food_id from USDA_ERS_staging.Food_32_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 33 as food_id from USDA_ERS_staging.Food_33_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 34 as food_id from USDA_ERS_staging.Food_34_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 35 as food_id from USDA_ERS_staging.Food_35_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 36 as food_id from USDA_ERS_staging.Food_36_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 37 as food_id from USDA_ERS_staging.Food_37_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 38 as food_id from USDA_ERS_staging.Food_38_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 39 as food_id from USDA_ERS_staging.Food_39_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 40 as food_id from USDA_ERS_staging.Food_40_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 41 as food_id from USDA_ERS_staging.Food_41_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 42 as food_id from USDA_ERS_staging.Food_42_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 43 as food_id from USDA_ERS_staging.Food_43_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 44 as food_id from USDA_ERS_staging.Food_44_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 45 as food_id from USDA_ERS_staging.Food_45_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 46 as food_id from USDA_ERS_staging.Food_46_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 47 as food_id from USDA_ERS_staging.Food_47_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 48 as food_id from USDA_ERS_staging.Food_48_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 49 as food_id from USDA_ERS_staging.Food_49_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 50 as food_id from USDA_ERS_staging.Food_50_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 51 as food_id from USDA_ERS_staging.Food_51_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 52 as food_id from USDA_ERS_staging.Food_52_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 53 as food_id from USDA_ERS_staging.Food_53_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 54 as food_id from USDA_ERS_staging.Food_54_Market;

Executing query with job ID: a4acc0eb-4bc8-443e-8081-51fa672ca5ea
Query executing: 2.37s

Conflict: 409 GET https://www.googleapis.com/bigquery/v2/projects/responsive-cab-267123/queries/a4acc0eb-4bc8-443e-8081-51fa672ca5ea?timeoutMs=400&location=US&maxResults=0: Already Exists: Table responsive-cab-267123:USDA_ERS_modeled.Food_Market

### Create Product_Food_Map Table 

In [ ]:
%%bigquery
CREATE TABLE Product_Food_Map
from
select food_id, foo from USDA_ERS_modeled.food


## Check Primary Keys

## Food_Market Table

food_market_id

In [ ]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Food_Market

In [ ]:
%%bigquery
select count(distinct food_market_id) as no_PKs from USDA_ERS_modeled.Food_Market

## Foods Table

In [ ]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Foods

In [ ]:
%%bigquery
select count(distinct food_id) as no_PKs from USDA_ERS_modeled.Foods

## Food_Categories Table

In [ ]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Food_Categories

In [ ]:
%%bigquery
select count(distinct category_id) as no_PKs from USDA_ERS_modeled.Food_Categories

## Market_Groups Table

In [6]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Market_Groups

,no_entries
0,39


In [7]:
%%bigquery
select count(distinct market_id) as no_PKs from USDA_ERS_modeled.Market_Groups

,no_PKs
0,39


# Foreign Key Check

market_id

In [14]:
%%bigquery
select g.market_id
from USDA_ERS_modeled.Food_Market fm left join USDA_ERS_modeled.Market_Groups g
on fm.market_id= g.market_id left join USDA_ERS_modeled.Geo_Market gm on gm.market_id = g.market_id
where fm.market_id is null

,market_id


food_id

In [15]:
%%bigquery
select fm.food_id
from USDA_ERS_modeled.Food_Market fm left join USDA_ERS_modeled.Foods f
on fm.food_id= f.food_id 
where f.food_id is null

,food_id


food_category

In [16]:
%%bigquery
select f.food_category
from USDA_ERS_modeled.Food_Categories fc left join USDA_ERS_modeled.Foods f
on fc.category_id= f.food_category
where fc.category_id is null

,food_category


# Predicting 2017 Food Prices Using Linear Regression in Apache Beam 
The transformed table will contain the food_id and the predicted 2017 price based on linear regression

In [2]:
%run Food_Market_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'USDA_ERS_modeled'
 projectId: 'responsive-cab-267123'
 tableId: 'Food_Market'> referenced by query SELECT food_id, year, AVG(price) as avg_price FROM USDA_ERS_modeled.Food_Market WHERE price IS NOT NULL and year IS NOT NULL GROUP BY food_id, year ORDER BY food_id limit 100
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.10 seconds.
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with n

In [3]:
%run Food_Market_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))
INFO:apache_beam.runners.dataflow.internal.apiclient:Starting GCS upload to gs://bmease_cs327e/staging/transform-foodmarket-df1.1588280104.581203/pipeline.pb...
INFO:apache_beam.runners.dataflow.internal.apiclient:Completed GCS upload to gs://bmease_cs327e/staging/transform-foodmarket-df1.1588280104.581203/pipeline.pb in 0 seconds.
INFO:apache_beam.runners.portability.stager:Downloading source distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/home/jupyter/venv/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmpn5xg5ms2', 'apache-beam==2.19.0', '--no-deps', '--no-binary', ':all:']
INFO:apache_beam.runners.portability.stager:Staging SDK sources from PyPI to gs://bmease_cs327e/staging/transform-foodmarket-df1.1588

# Primary Key Check for Food_Market_Beam_DF

In [9]:
%%bigquery
SELECT count(*) FROM USDA_ERS_modeled.Food_Market_Beam_DF

,f0_
0,430


In [10]:
%%bigquery
SELECT count(*) FROM (SELECT distinct food_id, year FROM USDA_ERS_modeled.Food_Market_Beam_DF)

,f0_
0,430


# Foreign Key Check for Food_Market_Beam_DF

In [11]:
%%bigquery
SELECT fm.food_id
FROM USDA_ERS_modeled.Food_Market_Beam_DF fm
LEFT OUTER JOIN USDA_ERS_modeled.Foods f
ON fm.food_id = f.food_id
WHERE f.food_id IS NULL

,food_id


In [11]:
%run Food_Map_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))
INFO:apache_beam.runners.dataflow.internal.apiclient:Starting GCS upload to gs://bmease_cs327e/staging/foodmap-df.1588128899.077566/pipeline.pb...
INFO:oauth2client.transport:Refreshing due to a 401 (attempt 1/2)
INFO:oauth2client.transport:Refreshing due to a 401 (attempt 2/2)
INFO:oauth2client.transport:Refreshing due to a 401 (attempt 1/2)
INFO:apache_beam.runners.dataflow.internal.apiclient:Completed GCS upload to gs://bmease_cs327e/staging/foodmap-df.1588128899.077566/pipeline.pb in 0 seconds.
INFO:apache_beam.runners.portability.stager:Downloading source distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/home/jupyter/venv/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmpb8niz0rx', 'apache-beam==2.19.0', 

In [13]:
%run Food_Map_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:oauth2client.transport:Refreshing due to a 401 (attempt 1/2)
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'instacart_modeled'
 projectId: 'responsive-cab-267123'
 tableId: 'Products'> referenced by query select lower(product_name) as product_name, product_id, a.aisle_id, department from instacart_modeled.Products p inner join instacart_modeled.Departments d on d.department_id = p.department_id inner join instacart_modeled.Aisles a on a.aisle_id = p.aisle_id where d.department_id not in (5,8,11,17,18) and p.product_name not like '%Filters%' order by product_name limit 100
INF

# Primary Key Check for Food_Map_Beam_DF

In [21]:
%%bigquery
SELECT count(*) from USDA_ERS_modeled.Food_Map_Beam_DF

,f0_
0,36907


In [22]:
%%bigquery
SELECT count(*) from (SELECT distinct food_id, product_id from USDA_ERS_modeled.Food_Map_Beam_DF)

,f0_
0,36907


# Foreign Key Check for Food_Map_Beam_DF

In [23]:
%%bigquery
SELECT m.food_id
FROM USDA_ERS_modeled.Food_Map_Beam_DF m
LEFT OUTER JOIN USDA_ERS_modeled.Foods f
ON m.food_id = f.food_id
WHERE f.food_id IS NULL

,food_id


In [24]:
%%bigquery
SELECT m.product_id
FROM USDA_ERS_modeled.Food_Map_Beam_DF m
LEFT OUTER JOIN instacart_modeled.Products p
ON m.product_id = p.product_id
WHERE p.product_id IS NULL

,product_id
